In [1]:
from main import DataImporter

data_importer = DataImporter()
data_importer.import_data()

ds_train = data_importer.get_train_data()
ds_test = data_importer.get_test_data()
ds_validation = data_importer.get_validation_data()

display(ds_train.head())
display(ds_test.head())
display(ds_validation.head())

,id,label,statement,subject,speaker,job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context
0,2635.json,0,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,2,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,0,Health care reform legislation is likely to ma...,health-care,blog-posting,,,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,1,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


,id,label,statement,subject,speaker,job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context
0,11972.json,3,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30.0,30.0,42.0,23.0,18.0,Radio interview
1,11685.json,0,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2.0,1.0,0.0,0.0,0.0,a news conference
2,11096.json,0,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63.0,114.0,51.0,37.0,61.0,comments on ABC's This Week.
3,5209.json,1,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1.0,1.0,3.0,1.0,1.0,a radio show
4,9524.json,5,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,,Wisconsin,democrat,5.0,7.0,2.0,2.0,7.0,a web video


,id,label,statement,subject,speaker,job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context
0,12134.json,4,We have less Americans working now than in the...,"economy,jobs",vicky-hartzler,U.S. Representative,Missouri,republican,1.0,0.0,1.0,0.0,0.0,an interview with ABC17 News
1,238.json,5,"When Obama was sworn into office, he DID NOT u...","obama-birth-certificate,religion",chain-email,,,none,11.0,43.0,8.0,5.0,105.0,
2,7891.json,0,Says Having organizations parading as being so...,"campaign-finance,congress,taxes",earl-blumenauer,U.S. representative,Oregon,democrat,0.0,1.0,1.0,1.0,0.0,a U.S. Ways and Means hearing
3,8169.json,1,Says nearly half of Oregons children are poor.,poverty,jim-francesconi,Member of the State Board of Higher Education,Oregon,none,0.0,1.0,1.0,1.0,0.0,an opinion article
4,929.json,1,On attacks by Republicans that various program...,"economy,stimulus",barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,interview with CBS News


In [2]:
ds_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10269 entries, 0 to 10268
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10269 non-null  object 
 1   label                 10269 non-null  int64  
 2   statement             10269 non-null  object 
 3   subject               10269 non-null  object 
 4   speaker               10269 non-null  object 
 5   job_title             10269 non-null  object 
 6   state_info            10269 non-null  object 
 7   party_affiliation     10269 non-null  object 
 8   barely_true_counts    10269 non-null  float32
 9   false_counts          10269 non-null  float32
 10  half_true_counts      10269 non-null  float32
 11  mostly_true_counts    10269 non-null  float32
 12  pants_on_fire_counts  10269 non-null  float32
 13  context               10269 non-null  object 
dtypes: float32(5), int64(1), object(8)
memory usage: 922.7+ KB


In [4]:
ds_train_clean = ds_train[['statement', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts']]
ds_train_clean.head()

,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts
0,Says the Annies List political group supports ...,0.0,1.0,0.0,0.0,0.0
1,When did the decline of coal start? It started...,0.0,0.0,1.0,1.0,0.0
2,"Hillary Clinton agrees with John McCain ""by vo...",70.0,71.0,160.0,163.0,9.0
3,Health care reform legislation is likely to ma...,7.0,19.0,3.0,5.0,44.0
4,The economic turnaround started at the end of ...,15.0,9.0,20.0,19.0,2.0


In [6]:
ds_train_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10269 entries, 0 to 10268
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   statement             10269 non-null  object 
 1   barely_true_counts    10269 non-null  float32
 2   false_counts          10269 non-null  float32
 3   half_true_counts      10269 non-null  float32
 4   mostly_true_counts    10269 non-null  float32
 5   pants_on_fire_counts  10269 non-null  float32
dtypes: float32(5), object(1)
memory usage: 280.9+ KB
